# Universal Transverse Mercator coordinate system

In [1]:
import utm

import numpy as np
import pandas as pd

import folium
from folium import plugins

import time
import random
from statistics import mean, median
from math import radians, cos, sin, asin, sqrt
import os

ModuleNotFoundError: No module named 'utm'

In [3]:
!pip install utm

You are using pip version 10.0.1, however version 20.3.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
from auxiliary_functions import *

We try the UTM package on GeoLife dataset

In [5]:
# GEOLIFE_DATA_PATH =r"C:\Users\Tay\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"
GEOLIFE_DATA_PATH =r"C:\Users\tay.yq.XTRAMAN\Documents\GitHub\Trajectory-Data-Mining\Geolife Trajectories 1.3\Data"

In [6]:
users = os.listdir(GEOLIFE_DATA_PATH)
print("Number of users in Geolife dataset: " + str(len(users)))

Number of users in Geolife dataset: 182


In [7]:
def readUserTraj(path_to_user):
    '''
    :param path_to_user: a path to the user's dataset according to the Geolife dataset file system
    :return: a dataframe containing all of the user's trajectories
    '''
    trajs = os.listdir(path_to_user + r"\Trajectory")
    col_names = ["lat", "lon", "alt", "date", "time"]
    userTraj = pd.DataFrame(columns=["lat", "lon", "alt", "time"])
    
    for i in range(len(trajs)):
        TRAJ_PATH = path_to_user + r"\Trajectory" + r"\\" + trajs[i]
        traj = pd.read_csv(TRAJ_PATH, skiprows=6, header=None, usecols=[0,1,3,5,6], names=col_names)
        
        traj["full_date"] = traj["date"].str.cat(traj["time"], sep=" ")
        traj["full_date"] = traj["full_date"].apply(lambda x: pd.to_datetime(x))

        traj = traj.drop(columns = ["date", "time"])
        traj = traj.rename(columns = {"full_date": "time"})
        
        userTraj = userTraj.append(traj)
        userTraj = userTraj.reset_index(drop=True)
    
    return userTraj

In [8]:
%%time
trajDataFrames = [readUserTraj(GEOLIFE_DATA_PATH + r"\\" + users[i]) for i in range(5)]

Wall time: 2min 4s


**Some useful functions**

In [9]:
utm.from_latlon(51.2, 7.5)

(395201.31038112973, 5673135.241182375, 32, 'U')

In [13]:
utm.to_latlon(395201.31038112973, 5673135.241182375, 32, 'U')

(51.20000000399309, 7.499999999862897)

In [18]:
temp = trajDataFrames[0][["lat", "lon"]].to_numpy()
temp

array([[ 39.984702, 116.318417],
       [ 39.984683, 116.31845 ],
       [ 39.984686, 116.318417],
       ...,
       [ 40.000443, 116.327186],
       [ 40.000522, 116.327132],
       [ 40.000543, 116.327148]])

In [22]:
%%time
UTM_np = np.apply_along_axis(lambda x: utm.from_latlon(x[0], x[1]), 1, temp)
UTM_np

Wall time: 11.9 s


array([['441807.05662297236', '4426281.714311392', '50', 'S'],
       ['441809.8580366453', '4426279.583938533', '50', 'S'],
       ['441807.0430475015', '4426279.938450423', '50', 'S'],
       ...,
       ['442568.93811357446', '4428023.143781784', '50', 'T'],
       ['442564.3948573248', '4428031.946912262', '50', 'T'],
       ['442565.7782140618', '4428034.267425094', '50', 'T']],
      dtype='<U32')

In [50]:
UTM_df = pd.DataFrame(data=UTM_np)
UTM_df.columns = ["easting", "northing", "zone_number", "zone_letter"]
UTM_df

,easting,northing,zone_number,zone_letter
0,441807.05662297236,4426281.714311392,50,S
1,441809.8580366453,4426279.583938533,50,S
2,441807.0430475015,4426279.938450423,50,S
3,441804.3125899241,4426280.181319337,50,S
4,441793.86824508826,4426276.5982440645,50,S
...,...,...,...,...
173865,442574.79445660365,4428018.659661258,50,T
173866,442570.89301981183,4428022.01904634,50,T
173867,442568.93811357446,4428023.143781784,50,T
173868,442564.3948573248,4428031.946912262,50,T


In [51]:
UTM_df.zone_letter.unique()

array(['S', 'T', 'R'], dtype=object)

In [52]:
for i in range(1, len(UTM_df)):
    if UTM_df.zone_letter[i-1] != UTM_df.zone_letter[i]:
        print(i)

470
1152
1938
3390
3415
3424
3628
3631
3682
5847
6137
7614
7768
7769
7833
7849
7852
7853
7858
8154
8157
8423
8677
8698
8714
8716
8728
8885
8926
8928
8934
9153
9161
9386
9396
9542
9550
9551
9557
9739
9751
9811
9832
9835
9853
10292
10295
10300
10439
10508
10520
10714
10940
10941
10943
10944
10945
10946
10955
11211
11219
11274
11385
11629
11632
11653
11667
11740
11745
12077
12163
12174
12179
12231
12233
12327
12362
12452
12577
12578
12580
12581
12656
12700
12701
12703
12710
12717
12727
12850
12942
12943
12953
13564
13565
13567
13572
13576
13577
13588
13592
13606
13622
13623
13773
13784
13807
13808
13813
14036
14047
14058
14090
14381
14678
14679
14695
14943
14946
14956
14962
15165
15720
15872
15882
15889
15893
15904
15915
16368
16413
16440
16460
16467
16471
16482
16504
16874
16901
17361
17384
17393
17397
18106
18169
18172
18754
19211
19820
27553
29083
29424
31318
31903
31904
31951
31959
31965
31978
31982
31985
32018
32020
32059
35273
35693
36141
36817
37263
37559
37581
37587
37864
37873
37

In [53]:
UTM_df.iloc[460:480, :]

,easting,northing,zone_number,zone_letter
460,442338.7471803688,4427926.873683271,50,S
461,442342.71374168317,4427920.849734608,50,S
462,442341.52709383355,4427921.968706575,50,S
463,442337.0458428309,4427927.663562783,50,S
464,442347.4649654593,4427928.250593063,50,S
465,442350.76462077076,4427924.340662985,50,S
466,442354.65633569704,4427930.971052773,50,S
467,442357.95724282204,4427938.493903998,50,S
468,442364.7537436889,4427945.435286278,50,S
469,442372.04306259315,4427949.819989162,50,S
